In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pyodbc
import time

def executar_conexao_bd():
    server = 'TBDBL-GLEISSON'
    database = 'Dados_Pavimento'
    username = 'gleisson.santos'
    password = '12345'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

Executado a conexão com o BD......


In [9]:
#Iniciando modulos
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
import pandas as pd
import pyodbc
import time
import re
import os
import csv

#Acesso ao banco de dados Casa
def executar_conexao_bd():
    server = 'DESKTOP-0P0H9IO'
    database = 'Dados_Pavimento'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server inexistente ou acesso negado. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (53)')

In [2]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.80M/6.80M [00:01<00:00, 4.24MB/s]
C:\Users\GLEISS~1.SAN\AppData\Local\Temp/ipykernel_1628/3643764433.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
login = 't034183'
senha = 'CNB@2022'
driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
time.sleep(2)

# Procurar a tag random do site / Login e senha
randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(login)
driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
driver.find_element(By.ID, "loginForm-submit").click()
time.sleep(2)

In [4]:
#tipo: insu, menu: GOSI_anchor - ACESSAR MENU DE INSUMOS
def arvore_acesso(driver, tipo, menu ):
    # Proc detalhe da Os no campo de buscar
    driver.find_element(By.ID, "arvoreSearch").send_keys(tipo)
    # aguardar ate o elemento aparecer e/ou seguir codigo
    wait = WebDriverWait(driver, 5)
    element1 = wait.until(EC.element_to_be_clickable((By.ID, menu)))
    # acessar Campo detalhe da Os
    driver.find_element(By.ID, menu).click()
    time.sleep(3)
    #acessar camada frame do site
    driver.switch_to.frame("frame-content")

#ACESSAR MENU PAR EXTRAR INSUMADOS
arvore_acesso(driver, 'detalhe', 'DOSE_anchor')

####  Acesso ao item "Detalhes da Os"

In [5]:
#Consultar na label Material
def preencher_ss(driver, lista, lista2):
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt71-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt79-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt71-item"]').send_keys(lista)
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt79-item"]').send_keys(lista2)
    time.sleep(0.5)
    # clinical em buscar
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-panel"]/input').click()
    #coletar valor solicitado
    
    try:
        material = driver.find_element(By.CSS_SELECTOR, "label[title='Material Utilizado'] ").text
    except NoSuchElementException:
        material = 'None'
    
    try:
        hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do HD'] ").text
    except NoSuchElementException:
        try:
            hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do HD Instalado'] ").text
        except NoSuchElementException:
            try:
                hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do Hidrômetro Instalado'] ").text
            except NoSuchElementException:
                try:
                    hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do Hidrômetro'] ").text
                except NoSuchElementException:
                    try:
                        hd = driver.find_element(By.CSS_SELECTOR, "label[title='Hidrômetro Instalado'] ").text
                    except NoSuchElementException:
                        try:
                            hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do Hidrômetro para Religação'] ").text
                        except NoSuchElementException:
                            hd = None
                        
    return material, hd


In [6]:
df = pd.read_sql('SELECT * FROM [Dados HDs]', executar_conexao_bd())
#df = pd.DataFrame(df, columns = ['SS', 'OS','Serviço', 'Material', 'Mureta','Hd'])
df = pd.DataFrame(df, columns = ['SS', 'OS','Serviço','Hd'])
#df = df.astype(str)

#df = df.query("Tipo == '21 - LIGACAO DE AGUA'")

display(df)
n_linhas = len(df)
print("Número de linhas: ", n_linhas)

Executado a conexão com o BD......


,SS,OS,Serviço,Hd
0,904190044,1,11 - INSTALACAO DE HIDROM.,None
1,983464715,1,12 - SUBSTITUICAO HID.,None
2,983464721,1,12 - SUBSTITUICAO HID.,None
3,983464751,1,12 - SUBSTITUICAO HID.,None
4,983528741,1,12 - SUBSTITUICAO HID.,None
...,...,...,...,...
880,982814138,1,671 - SUBSTITUICAO HID COD 16,None
881,981788365,1,671 - SUBSTITUICAO HID COD 16,None
882,981692846,2,671 - SUBSTITUICAO HID COD 16,None
883,982447281,2,671 - SUBSTITUICAO HID COD 16,None


Número de linhas:  885


In [7]:
#Extrair valores e criar coluna Material insumado
dados = []
hds = []
for i, row in df.iterrows():
    a = preencher_ss(driver, row['SS'], row['OS'])
    time.sleep(0.3)
    dados.append(a[0])
    hds.append(a[1])
df['Material'] = dados
df['Hd'] = hds


In [8]:
display(df)

,SS,OS,Serviço,Hd,Material
0,904190044,1,11 - INSTALACAO DE HIDROM.,A18S899900,
1,983464715,1,12 - SUBSTITUICAO HID.,Y23L130001,
2,983464721,1,12 - SUBSTITUICAO HID.,Y23L128361,"1 HIDRO VZ 1,5M 1 LACRE AZUL FIO DE AÇO"
3,983464751,1,12 - SUBSTITUICAO HID.,A22HW0191480,1 HIDRO VZ 3M. 1 LACRE AZUL FIO DE AÇO.
4,983528741,1,12 - SUBSTITUICAO HID.,Y23L130004,"1 HD VZ 1,5 - 1 LACRE FIO DE AÇO - 1 REGISTRO ..."
...,...,...,...,...,...
880,982814138,1,671 - SUBSTITUICAO HID COD 16,A22HW0246987,1 hidro vz 3m 1 lacre azul fio de aço 1 par de...
881,981788365,1,671 - SUBSTITUICAO HID COD 16,A22HW0197492,3 joelho liso pvc DN 20mm 1 adaptador pvc de D...
882,981692846,2,671 - SUBSTITUICAO HID COD 16,A22SG0076365,1 HIDRO VZ 3M 1 LACRE AZUL FIO DE AÇO 1 PAR DE...
883,982447281,2,671 - SUBSTITUICAO HID COD 16,A22SG0083027,1 HD VZ 3 1 LACRE FIO DE AÇO


In [9]:
#Acessar tabela do BD e extrair apenas item do texto
m_encontrado = ['Mureta', 'MURETA','mureta']

material = []

for i, row in df.iterrows():
    if any(x in row['Material'] for x in m_encontrado):
        material.append('Sim')
    else:
        material.append('Não')     
df['Mureta'] = material


df = df.astype(str)
df = pd.DataFrame(df, columns = ['SS', 'OS', 'Material', 'Mureta','Hd'])
df_sim = df.loc[df['Mureta'] == 'Sim']
display(df_sim)


,SS,OS,Material,Mureta,Hd
831,982681194,2,7 METROS DE TUBO PEAD DN 20MM 1 COLAR DE TOMAD...,Sim,Y23L128465
832,982682419,2,7 METROS DE TUBO PEAD DN 20MM 1 COLAR DE TOMAD...,Sim,Y23L128464
833,982680577,2,7 METROS DE TUBO PEAD DN 20MM 1 COLAR DE TOMAD...,Sim,Y23L128463
834,983501529,2,7 METROS DE TUBO PEAD DN 20MM 1 COLAR DE TOMAD...,Sim,Y23L128466
838,983392412,2,1 mureta 1 colar de tomada pead dn 110x1/2. 1 ...,Sim,A22HW0183899
839,983391646,2,1 mureta 3 metros de tubo pead dn 20mm 1 colar...,Sim,A22HW0239337
840,983392818,2,1 MURETA 1 HIDRO VZ 3M 1 LACRE AZUL FIO DE AÇO...,Sim,A22HW0239331
843,983159859,2,1 COLAR DE TOMADA PEAD DN 63X1/2. 1 MURETA 3 M...,Sim,A22HW0246988
853,982913057,2,5 METROS DE TUBO PEAD DN 20MM 1 T DESERVIÇO PE...,Sim,Y23L104131
859,981679802,1,1 COLAR DE TOMADA FERRO DN 150X1/2. 6 METROS D...,Sim,Y23L104569


In [10]:
for index, row in df.iterrows():    
    ID = row.Material
    SS = row.SS
    MT = row.Mureta
    HD = row.Hd
    #comando_sql = (f"UPDATE [Janeiro26/12-25/01] SET Material='{ID}', Mureta='{MT}', Hd='{HD}' WHERE SS={SS} ")
    #comando_sql = "UPDATE [700_Lauro] SET Material='{}', Mureta='{}', Hd='{}' WHERE SS={}".format(
    #ID.replace("'", "''"), MT.replace("'", "''"), HD.replace("'", "''"), SS)  
    comando_sql = "UPDATE [Dados HDs] SET Hd='{}' WHERE SS={}".format(
    HD.replace("'", "''"), SS)  
    cursor.execute(comando_sql)
    
cursor.commit()

In [2]:
#Limpar colunas do Banco de dados;
#delete_query = f"DELETE FROM {table_name}"
#df = df.astype(str)
#df = pd.DataFrame(df, columns=['SS', 'OS', 'Matutilizado', 'Hdutilizado', 'Usoumureta'])

#cursor = executar_conexao_bd().cursor()
#for index, row in df.iterrows():
    #Mat = row.Matutilizado
    #HD  = row.Hdutilizado
    #INS = row.Usoumureta
    #SS  = row.SS
    #comando_sql = (f"UPDATE Material SET Matutilizado='NULL', Hdutilizado='NULL', Usoumureta='NULL' WHERE SS ={SS}")
    #cursor.execute(comando_sql)
    
#cursor.commit()
#cursor.close()